# SV Live Demo

In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Speaker Verification

### Model Load

In [2]:
from ResNet34 import ResNet34, ResNet34_v1

config = dict(
    loss="softmax",
    gpu_no=[0], no_cuda=True
)

model = ResNet34(config, inplanes=16, n_labels=1759)

In [6]:
model.load_extractor("./model_best.pth.tar")
model.eval()

if not config['no_cuda']:
    model.cuda()

extractor loaded from ./model_best.pth.tar


###  Speaker Verification System


In [24]:
import librosa
from manage_audio import preprocess_audio
import numpy as np
import torch
import torch.nn.functional as F

class sv_system():
    def __init__(self, model, n_dims=40, feat_format='fbank'):
        self.speaker_models = dict()
        self.dct_filters = librosa.filters.dct(n_filters=n_dims, n_input=n_dims)
        self.model = model
        self.n_dims = n_dims
        self.feat_format = feat_format
        
        # for test
#         self.speaker_models = self._random_speaker_model()
    
    def enrol(self, wav, spk_name):
        feat = self._wav2feat(wav)
        dvector = self._extract_dvector(feat).squeeze()
        if spk_name not in self.speaker_models:
            self.speaker_models[spk_name] = [dvector]
        else:
            self.speaker_models[spk_name] += [dvector]
        
    
    def _extract_dvector(self, feat):
        """
            dvector: ndarray
        """
        if feat.dim() == 2:
            feat = feat.unsqueeze(0).unsqueeze(0)
        
        dvector = self.model.embed(feat).detach().cpu().numpy()
        
        return dvector
    
    def _wav2feat(self, wav):
        """
            extracting input feature from wav (mfcc, fbank)
        """
        wav_data = librosa.core.load(wav, sr=16000)[0]
        feat = preprocess_audio(wav_data, n_mels=self.n_dims, 
                    dct_filters=self.dct_filters, in_feature=self.feat_format)
        
        return feat
        
    def _random_speaker_model(self):
        """
            random speaker model used for testing a sv_system
        """
        random_dvector = np.random.rand(4, self.model.embed_dim)
        random_speakers = ["a", "b", "c", "d"]

        random_speaker_models = dict.fromkeys(random_speakers)

        for i, key in enumerate(random_speaker_models.keys()):
            random_speaker_models[key] = [random_dvector[i]]*2
        
        return random_speaker_models
    
    def init_speaker_model(self):
        self.speaker_model = dict()
    
    def verify(self, wav):
        """
            verify a input wav and output a verification result
            and rank-1 identification
        """
        feat = self._wav2feat(wav)
        test_dvector = self._extract_dvector(feat)
        # order keep?
        # averaging all dvectors for each speaker
        avg_speaker_models = np.stack([np.mean(v, axis=0) for v in self.speaker_models.values()], 
                                      axis=0)

        score = F.cosine_similarity(torch.from_numpy(avg_speaker_models).float(), 
                                    torch.from_numpy(test_dvector).float(), dim=1)
        threshold = 0.607
        pred_speaker = list(self.speaker_models.keys())[torch.argmax(score)]
        if max(score) > threshold:
            print("Accepted as {} ({:.3f})".format(pred_speaker, max(score)))
        else:
            print("Reject ({:.3f})".format(max(score)))
            
            
        

In [12]:
test_sv_system = sv_system(model)

### Enrollment

In [18]:
test_sv_system.enrol("test_wavs/f92e49f3_nohash_2.wav", 'ID1')

In [20]:
test_sv_system.enrol("test_wavs//b959cd0c_nohash_1.wav", 'ID2')

### Verification &  Identification

In [21]:
test_sv_system.verify("test_wavs/b959cd0c_nohash_3.wav")

Accepted as ID2 (0.780)


In [22]:
test_sv_system.verify("test_wavs/063d48cf_nohash_0.wav")

Reject (0.372)
